In [26]:
from datetime import date, datetime
import time
from urllib.parse import urlencode
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import date
import re
import json
from multiprocessing.dummy import Pool
from tqdm import tqdm
import sys

headers2 = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36'}
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
base_urls = ['https://www.booking.com/searchresults.ru.html', 'https://travel.yandex.ru/hotels/']
max_records = 50
records_per_page = 25

def numToMonth(num):
    return {
            1: 'Январь',
            2: 'Февраль',
            3: 'Март',
            4: 'Апрель',
            5: 'Май',
            6: 'Июнь',
            7: 'Июль',
            8: 'Август',
            9: 'Сентябрь', 
            10: 'Октябрь',
            11: 'Ноябрь',
            12: 'Декабрь'
    }[num]



def generateLink(dest: str, checkin: datetime, checkout: datetime, base_url: str):
    if(base_url.__contains__('booking')):
        base_url += "?" + urlencode(
            {
                "ss": dest,
                "checkin_year": checkin.year,
                "checkin_month": checkin.month,
                "checkin_monthday": checkin.day,
                "checkout_year": checkout.year,
                "checkout_month": checkout.month,
                "checkout_monthday": checkout.day,
                "no_rooms": 1,
                "offset": 0
            }
        )
    elif(base_url.__contains__('travel.yandex')):
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver = webdriver.Chrome(options=option)
        driver.get(base_url)
        driver.maximize_window()

        city_input = driver.find_element(By.XPATH, "//input[@class='w_eHd input_center']")
        city_input.clear()
        city_input.send_keys(city)
        time.sleep(1)
        elem = driver.find_element(By.XPATH, "//div[@class='rC02U _PIrG sfzRD w5F6Y P0ZeS']")
        elem.click()
        time.sleep(1)
        #checkin input
        #open calendar
        #calendar_button = driver.find_element(By.XPATH, "//label[@class='ZWemG nZhti datePickerTrigger_viewType_union datePickerTrigger_type_startDate O_nYt JFpXj datePickerTrigger_viewType_union datePickerTrigger_type_startDate']")
        #calendar_button = driver.find_element(By.XPATH, "//button[@class='cD6tX']")
        #time.sleep(1)
        #calendar_button.click()
        #time.sleep(1)

        #load calendar
        months_list = driver.find_elements(By.XPATH, "//div[@class='monthsList']//div")
        months_list[-1].click()
        #looking for correct month index
        checkin_month_index = 0 #февраль - 7
        for i in range(0, 12):
            if(months_list[i].text.__contains__(numToMonth(checkin.month))):
                checkin_month_index = i
                break
        checkout_month_index = checkin_month_index + checkout.month - checkin.month
        time.sleep(1)

        month_counter = -1

        for day in driver.find_elements(By.XPATH, "//div[@class='mYiO8']/div/span"):
            if(day.text == "1"):
                month_counter = month_counter + 1
            if(month_counter == checkin_month_index and day.text == checkin.day.__str__()):
                day.click()
            if(month_counter == checkout_month_index and day.text == checkout.day.__str__()):
                day.click()
                break


        submit_button = driver.find_element(By.XPATH, "//button[@class='vHqxX z8gtM']")
        submit_button.click()

        time.sleep(2)
        base_url = driver.current_url
        driver.quit()

    return base_url

def getData(url_to_parse: str, hotels_amount: int = 25):
    hotel_cards = []
    hotel_counter = 0
    if(url_to_parse.__contains__('travel.yandex')):
        

        for i in range(0, max_records, records_per_page):
            url_to_parse = re.sub('navigationToken=\d+', 'navigationToken=' + i.__str__(), url_to_parse)
            url_to_parse = re.sub('-\d+-newsearch', f'-{round(i / 25) + 1}-newsearch', url_to_parse)
            
            option = webdriver.ChromeOptions()
            option.add_argument('headless')
            driver = webdriver.Chrome(options=option)
            driver.get(url_to_parse)
            driver.maximize_window()
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            driver.quit()

            for item in soup.find_all('div', 'ATR04'): #going through hotel records
                hotel_data = {}
                hotel_data['url'] = 'https://travel.yandex.ru' + item.find('a', 'Link Link_theme_normal Link_view_default STEEy Y_QIb Link_lego vW74J').get('href')
                hotel_data['name'] = item.find('a', 'Link Link_theme_normal Link_view_default STEEy Y_QIb Link_lego KlhYG').get_text()
                hotel_data['rate'] = item.find('span', 'Akpkj PotX_').get_text()
                try:
                    hotel_data['stars'] = item.find('span', 'd2q3W').find('span').get_text()
                except AttributeError:
                    hotel_data['stars'] = ''

                hotel_soup = BeautifulSoup(requests.get(hotel_data['url'],headers=headers).content, 'html.parser')
                hotel_data['address'] = hotel_soup.find('span', 'address Y7xGC').get_text().replace('\n', '')    #hotel address
                
                #coords
                imgsrc = ''
                images = hotel_soup.find_all('img')
                for img in images:
                    if img.has_attr('src'):
                        if img['src'].__contains__('static-maps.yandex.ru'):
                            imgsrc = img['src']
                for str in imgsrc.split('&'):
                    if(str.__contains__('ll=')):
                        coords = str.removeprefix('ll=').split('%2C')
                        hotel_data['latitude'] = coords[1]
                        hotel_data['longitude'] = coords[0]
                #tags
                hotel_data['tags'] = []
                for tag in hotel_soup.find_all('div', 'diui0 avQrE h865h'):
                    hotel_data['tags'].append(tag.find('span', 'G7iZ5').get_text().replace('\n', ''))


                for tag in hotel_soup.find_all('li', 'cKijb'):
                    hotel_data['tags'].append(tag.get_text().replace('\n', ''))

                print(hotel_counter)
                print(hotel_data)
                hotel_counter += 1
                hotel_cards.append(hotel_data)


    elif(url_to_parse.__contains__('booking')):
        #счетчик отелей
        item_counter = 1
        #отступ для url
        for i in range(0, max_records, records_per_page):   #переход по страницам
            #изменяю url на другую страницу
            url_to_parse = re.sub('offset=\d+', 'offset=' + i.__str__(), url_to_parse)
            #soup страницы с отелями
            soup = BeautifulSoup(requests.get(url_to_parse,headers=headers).content, 'html.parser')

            for item in soup.find_all('div', 'd20f4628d0'): #going through hotel records/переход по блокам отелей
                #пустой словарь для инфы отеля
                hotel_data = {}
                #just counting for myself
                print(item_counter)
                item_counter += 1

                #extract data
                hotel_data['name'] = item.find('div', 'fcab3ed991 a23c043802').get_text() #hotel name
                hotel_data['rate'] = item.find('span', 'fcab3ed991 bd73d13072').get_text() #price with discount
                hotel_data['url'] = item.find('a', 'e13098a59f').get('href') #hotel url
                temp_url = item.find('a', 'e13098a59f').get('href')
                try:
                    hotel_data['stars'] = (round(float(item.find('div', 'b5cd09854e d10a6220b4').get_text().replace(',','.')) / 2)).__str__() #hotel rating
                except AttributeError:
                    hotel_data['stars'] = ''

                #open hotel url to get more data
                hotel_soup = BeautifulSoup(requests.get(temp_url,headers=headers).content, 'html.parser')
                hotel_data['address'] = hotel_soup.find('span', 'hp_address_subtitle js-hp_address_subtitle jq_tooltip').get_text().replace('\n', '')    #address
                coords = hotel_soup.find('a', id='hotel_address').get('data-atlas-latlng').split(',') #hotel address
                hotel_data['latitude'] = coords[0]
                hotel_data['longitude'] = coords[1]
                tags = hotel_soup.find_all('div', 'important_facility')
                hotel_data['tags'] = []
                for tag in tags:
                    hotel_data['tags'].append(tag.get_text().replace('\n', ''))

                print(hotel_data)
                hotel_cards.append(hotel_data)
    return hotel_cards

####################################################################################

#генерирует ссылку для букинга
def generateLink(dest: str, checkin: datetime, checkout: datetime, base_url: str):
    new_url = base_url + "?" + urlencode(
        {
            "ss": dest,
            "checkin_year": checkin.year,
            "checkin_month": checkin.month,
            "checkin_monthday": checkin.day,
            "checkout_year": checkout.year,
            "checkout_month": checkout.month,
            "checkout_monthday": checkout.day,
            "no_rooms": 1,
            "offset": 0
        }
    )
    return new_url


#список ссылок на отели
def get_hotel_urls(url_to_parse: str, hotels_amount: int = 10): #for booking
    #empty url list
    hotel_urls = []
    #счетчик отелей
    item_counter = 0
    #отступ для url
    for i in range(0, max_records, records_per_page):   #переход по страницам
        #изменяю url на другую страницу
        url_to_parse = re.sub('offset=\d+', 'offset=' + i.__str__(), url_to_parse)
        #soup страницы с отелями
        soup = BeautifulSoup(requests.get(url_to_parse,headers=headers).content, 'html.parser')

        for item in soup.find_all('div', 'd20f4628d0'): #going through hotel records/переход по блокам отелей
            #extract url
            hotel_url = item.find('a', 'e13098a59f').get('href') #hotel url
            #add to list
            hotel_urls.append(hotel_url)
            #increment counter
            item_counter += 1
            #stop if enougth
            if(item_counter == hotels_amount):
                break
        if(item_counter == hotels_amount):
            break
    return hotel_urls

#извлекает данные об отеле по ссылке
def get_hotel_data(url):   #booking
    hotel_soup = BeautifulSoup(requests.get(url,headers=headers).content, 'html.parser')
    #name //*[@id="hp_hotel_name"]/text() 
    name = hotel_soup.find('h2', id='hp_hotel_name').get_text().replace('\n', '').removeprefix('Отель')
    #url is url
    #rate
    rate = hotel_soup.find('span', 'prco-valign-middle-helper').get_text().replace('\n', '')
    #stars
    try:
        stars = (round(float(hotel_soup.find('div', 'b5cd09854e d10a6220b4').get_text().replace(',','.')) / 2)).__str__() #hotel rating
    except AttributeError:
        stars = ''
    #address
    address = hotel_soup.find('span', 'hp_address_subtitle js-hp_address_subtitle jq_tooltip').get_text().replace('\n', '')
    #latitude
    #longitude
    coords = hotel_soup.find('a', id='hotel_address').get('data-atlas-latlng').split(',') #hotel address
    latitude = coords[0]
    longitude = coords[1]
    soup_tags = hotel_soup.find_all('div', 'important_facility')
    #tags
    tags = []
    for tag in soup_tags:
        tags.append(tag.get_text().replace('\n', ''))

    hotel_data = {
        'name': name,
        'url': url,
        'rate': rate,
        'stars': stars,
        'address': address,
        'latitude': latitude,
        'longitude': longitude,
        'tags': tags
    }
    return hotel_data

#мультипоточно парсит данные
#def get_all_hotel_data(urls, thread_number = 4):
    pool = Pool(thread_number)
    all_data = pool.map(get_hotel_data, urls)
    return all_data

def get_all_hotel_data(urls, thread_number = 4):
    pool = Pool(processes=thread_number)

    all_data = []
    for result in tqdm(pool.imap(func=get_hotel_data, iterable=urls), total=len(urls)):
        all_data.append(result)
    return all_data

#для сравнения
def get_all_hotel_data2(urls, thread_number = 4):
    all_data = []
    for item in urls:
        all_data.append(get_hotel_data(item))
    return all_data


city = "Стабмбул"
checkin = date(2022, 9, 10)   #YYYY-MM-DD
checkout = date(2022, 9, 11)
booking_url = generateLink(city, checkin, checkout, base_urls[0])
#yandex_url = generateLink(city, checkin, checkout, base_urls[1])
#print(yandex_url)
print(booking_url)



#hotel_cards = getData(booking_url)
#hotel_cards = getData(yandex_url)



https://www.booking.com/searchresults.ru.html?ss=%D0%A1%D1%82%D0%B0%D0%B1%D0%BC%D0%B1%D1%83%D0%BB&checkin_year=2022&checkin_month=9&checkin_monthday=10&checkout_year=2022&checkout_month=9&checkout_monthday=11&no_rooms=1&offset=0


In [28]:
urls = get_hotel_urls(booking_url, 25)

In [29]:
all_data = get_all_hotel_data(urls)

100%|██████████| 25/25 [00:39<00:00,  1.58s/it]


In [38]:
all_data = get_all_hotel_data2(urls)

In [31]:
for i in all_data:
    print(i)

{'name': 'Old Port Hotel Suit', 'url': 'https://www.booking.com/hotel/tr/old-port-suit-47-sultanahmet.ru.html?aid=304142&ucfs=1&arphpl=1&checkin=2022-09-10&checkout=2022-09-11&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=distance_from_search&srpvid=69a43c8187480206&srepoch=1657096581&all_sr_blocks=551826901_202211209_2_0_0&highlighted_blocks=551826901_202211209_2_0_0&matching_block_id=551826901_202211209_2_0_0&sr_pri_blocks=551826901_202211209_2_0_0__14400&from=searchresults#hotelTmpl', 'rate': '9 660\xa0руб.', 'stars': '5', 'address': 'Alemdar Mahallesi, Divanyolu Caddesi, Seftali Sokak, No:16 Fatih / Sultanahmet, Фатих, 34110 Стамбул, Турция', 'latitude': '41.00819500', 'longitude': '28.97743700', 'tags': ['Трансфер от/до аэропорта', 'Номера для некурящих', 'Бесплатный Wi-Fi', 'Wi-Fi доступен на всей территории', 'Ресторан', 'Кофеварка/чайник во всех номерах', 'Бар', 'Трансфер от/до аэропорта', 'Номера для некурящих', 'Бесплатный Wi-F

In [8]:
with open("data\\temp.json", "w", encoding='utf-8') as final:
    json.dump(all_data, final, ensure_ascii=False, indent=2)

In [3]:
hotel_cards = getData(booking_url)
#hotel_cards = getData(yandex_url)

1
{'name': 'Old Port Hotel Suit', 'rate': '8 368\xa0руб.', 'url': 'https://www.booking.com/hotel/tr/old-port-suit-47-sultanahmet.ru.html?aid=304142&ucfs=1&arphpl=1&checkin=2022-09-10&checkout=2022-09-11&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=distance_from_search&srpvid=73dd3600e4a500cf&srepoch=1657006850&all_sr_blocks=551826901_202211209_2_0_0&highlighted_blocks=551826901_202211209_2_0_0&matching_block_id=551826901_202211209_2_0_0&sr_pri_blocks=551826901_202211209_2_0_0__14400&from=searchresults#hotelTmpl', 'stars': '5', 'address': 'Alemdar Mahallesi, Divanyolu Caddesi, Seftali Sokak, No:16 Fatih / Sultanahmet, Фатих, 34110 Стамбул, Турция', 'latitude': '41.00819500', 'longitude': '28.97743700', 'tags': ['Трансфер от/до аэропорта', 'Номера для некурящих', 'Бесплатный Wi-Fi', 'Wi-Fi доступен на всей территории', 'Ресторан', 'Кофеварка/чайник во всех номерах', 'Бар', 'Трансфер от/до аэропорта', 'Номера для некурящих', 'Бесплатный Wi

Код для tutu.hotels

In [56]:
from selenium.webdriver.common.action_chains import ActionChains

tutu_url = 'https://hotel.tutu.ru/offers?business=0&check_in=2022-08-09&check_out=2022-08-10&contract=&filters=&geo_id=2657260&geo_name=Москва&geo_type=locality&room[0]=1.&search_id=8bddf625-9cba-4e69-84d8-d8fc25e1f616&sorting=relevanceDesc'
tutu_per_page = 20

def get_hotel_urls_tutu(url_to_parse: str, hotels_amount: int = 10): #for tutu в виде ленты
    #селениум
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    #driver = webdriver.Chrome(options=option)
    driver = webdriver.Chrome()
    driver.get(url_to_parse)
    driver.maximize_window()
    time.sleep(10)
    
    #загрузка необходимого кол-ва отелей
    for i in range(tutu_per_page, max_records, tutu_per_page):   #переход по страницам
        if(i >= hotels_amount): #остановиться когда достаточно
            break

        #переместиться вниз для загрузки новых отелей
        bottom_element = driver.find_element(By.XPATH, "//span[@class='aGfNlsl9rp91gLVoKHzCQ o33124 o33134']")
        actions = ActionChains(driver)
        actions.move_to_element(bottom_element).perform()
        time.sleep(5)
    
    item_counter = 0
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for i in soup.find_all('span', 'o33143 o33142 o33129'):
        print(i.get_text())
        item_counter += 1
        #stop if enougth
        if(item_counter == hotels_amount):
            break
        

In [57]:
get_hotel_urls_tutu(tutu_url)

Гостиница Метрополь
Гостиница Националь Москва
Отель The Ritz-Carlton, Moscow
Отель Mercure Арбат Москва
Арарат Парк Хаятт Москва
Отель Moscow Marriott Grand
AZIMUT Сити Отель Смоленская Москва
Отель Crowne Plaza Moscow World Trade Centre
Отель Балчуг Кемпински Москва
Лотте Отель Москва


Черновик

In [83]:
with open("data\hotel5.json", "a", encoding='utf-8') as final:
    json.dump(hotel_cards, final, ensure_ascii=False, indent=2)

In [4]:
with open('data\hotels.json', encoding='utf-8') as f:
   data = json.load(f)

print(data)

[{'name': 'Motel One Paris-Porte Dorée', 'rate': '8 554\xa0руб.', 'url': 'https://www.booking.com/hotel/fr/motel-one-paris-porte-doree.ru.html?aid=304142&ucfs=1&arphpl=1&checkin=2022-09-10&checkout=2022-09-11&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=9a3390498c1e00ee&srepoch=1656966676&all_sr_blocks=306580401_326766627_2_2_0&highlighted_blocks=306580401_326766627_2_2_0&matching_block_id=306580401_326766627_2_2_0&sr_pri_blocks=306580401_326766627_2_2_0__14900&from_sustainable_property_sr=1&from=searchresults#hotelTmpl', 'stars': '4', 'address': '295 Avenue Daumesnil, 12-й округ: Берси, 75012 Париж, Франция', 'latitude': '48.83494200', 'longitude': '2.41024600', 'tags': ['Номера для некурящих', 'Удобства для гостей с ограниченными физическими возможностями', 'Wi-Fi доступен на всей территории', 'Бесплатный Wi-Fi', 'Парковка (на территории)', 'Парковка', 'Бар', 'Очень хороший завтрак', 'Номера для некурящих', 'Удобства

In [103]:
driver = webdriver.Chrome()
driver.get('https://travel.yandex.ru/hotels/')
driver.maximize_window()

In [102]:
stok = 'https://ostrovok.ru/hotel/france/paris/?q=2734&dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&room=s-6ece6b45-b117-5ef8-bbbc-5296e5633868&serp_price=grand_hotel_dechampaigne.18833.RUB.h-acdf30be-a9a6-56ee-a8fb-4d58d6fecbc7&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1'
g = 100

stok = re.sub('navigationToken=\d+', 'navigationToken=' + g.__str__(), stok)
stok = re.sub('-\d+-newsearch', f'-{round(g / 25) + 1}-newsearch', stok)
print(stok)

https://travel.yandex.ru/hotels/search/?adults=2&bbox=37.421753000000024%2C55.627469722387445~37.756977000000006%2C55.847358&checkinDate=2022-07-11&checkoutDate=2022-07-12&childrenAges=&geoId=213&lastSearchTimeMarker=1656982649361&navigationToken=100&searchPagePollingId=b7464c6d76b0f0f8976788f17f68a33b-5-newsearch&selectedSortId=relevant-first


In [132]:
url4 = 'https://ostrovok.ru/hotel/france/paris/?q=2734&dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&room=s-6ece6b45-b117-5ef8-bbbc-5296e5633868&serp_price=grand_hotel_dechampaigne.18833.RUB.h-acdf30be-a9a6-56ee-a8fb-4d58d6fecbc7&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1'

option = webdriver.ChromeOptions()
option.add_argument('headless')
driver2 = webdriver.Chrome()
driver2.get(url4)
driver2.maximize_window()
time.sleep(8)



soup4 = BeautifulSoup(driver2.page_source, 'html.parser')

In [133]:
for item in driver2.find_elements(By.XPATH, "//a[@class='zen-hotelcard-nextstep-button button-view-primary button-size-s']"):
    print(item.get_attribute('href'))

https://ostrovok.ru/hotel/france/paris/mid7576717/novotel_paris_les_halles_2/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=novotel_paris_les_halles_2.39526.RUB.h-5122308b-45a3-58cd-98f2-7d19dac48c4e&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-57f34ae8-a78e-5582-964f-b7d3986ee97c
https://ostrovok.ru/hotel/france/paris/mid7576728/novotel_paris_tour_eiffel/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=novotel_paris_tour_eiffel.17290.RUB.h-3c1468ec-7186-56d7-8b81-fe6c89ab5b44&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-7fc56185-29ca-53a3-bfef-796d3322c723
https://ostrovok.ru/hotel/france/paris/mid7345596/campanile_paris_sud__porte_ditalie/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=campanile_paris_sud__porte_ditalie.6940.RUB.h-ef7e33b8-f20a-53b5-bf89-cc63e9b5e339&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-a9940f50-88ea-514b-92d4-78d925acd804
https://ostrovok.ru/hotel/france/paris/mid7576040/merc

In [134]:
for item in soup4.find_all('a', 'zen-hotelcard-nextstep-button button-view-primary button-size-s'):
    print('https://ostrovok.ru' + item.get('href'))

https://ostrovok.ru/hotel/france/paris/mid7576717/novotel_paris_les_halles_2/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=novotel_paris_les_halles_2.39526.RUB.h-5122308b-45a3-58cd-98f2-7d19dac48c4e&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-57f34ae8-a78e-5582-964f-b7d3986ee97c
https://ostrovok.ru/hotel/france/paris/mid7576728/novotel_paris_tour_eiffel/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=novotel_paris_tour_eiffel.17290.RUB.h-3c1468ec-7186-56d7-8b81-fe6c89ab5b44&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-7fc56185-29ca-53a3-bfef-796d3322c723
https://ostrovok.ru/hotel/france/paris/mid7345596/campanile_paris_sud__porte_ditalie/?dates=09.07.2022-10.07.2022&guests=2&page=1&price=one&q=2734&serp_price=campanile_paris_sud__porte_ditalie.6940.RUB.h-ef7e33b8-f20a-53b5-bf89-cc63e9b5e339&sid=8f8ccfd9-fcbe-4584-b086-7a5c413749f1&room=s-a9940f50-88ea-514b-92d4-78d925acd804
https://ostrovok.ru/hotel/france/paris/mid7576040/merc